# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import and show csv file
csv_file = "../Output/city_data.csv"
city_data = pd.read_csv(csv_file)

city_data.head()


,City,Country,Latitude,Longtitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,port alfred,za,-84.993006,48.938688,66.00,97,0,1.01
1,ushuaia,ar,-86.603767,-69.181265,47.46,57,100,14.94
2,souillac,mu,-47.684674,72.382445,76.50,83,75,11.41
3,chuy,uy,-45.453788,-42.502154,76.30,69,45,7.72
4,sao felix do xingu,br,-7.798523,-52.246315,80.53,89,100,3.53


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Set parameters for heat layer
locations = city_data[["Latitude", "Longtitude"]]
humidity = city_data["Humidity (%)"]

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create map
fig = gmaps.figure()

# Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max(humidity), dissipating=False, point_radius=2)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities based on speciic criteria
vacation_cities = city_data.loc[(city_data["Temperature (F)"] >= 70) &\
                                (city_data["Temperature (F)"] <= 85) &\
                                (city_data["Wind Speed (mph)"] < 10) &\
                                (city_data["Cloudiness (%)"] == 0), :]

vacation_cities


,City,Country,Latitude,Longtitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
14,guasave,mx,25.516607,-108.557260,76.64,66,0,3.00
30,east london,za,-66.912550,48.225462,77.31,85,0,0.51
43,guerrero negro,mx,12.890901,-130.441015,71.26,43,0,4.36
102,alice springs,au,-26.499019,135.364132,71.15,49,0,7.87
122,marawi,sd,19.028539,32.922236,77.00,91,0,1.01
124,auki,sb,-2.729819,163.718290,83.41,76,0,6.08
141,ghatanji,in,20.043351,78.511987,70.07,56,0,3.33
174,castro,cl,-50.276241,-89.312267,77.29,53,0,0.98
227,ballina,au,-31.149316,158.836065,73.99,97,0,5.50
282,kirakira,sb,-15.131528,162.232037,80.76,78,0,3.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create new dataframe
hotel_df = pd.DataFrame({"City": vacation_cities["City"],
                         "Country": vacation_cities["Country"],
                         "Latitude": vacation_cities["Latitude"],
                         "Longtitude": vacation_cities["Longtitude"],
                         "Hotel Name": ""})

hotel_df

,City,Country,Latitude,Longtitude,Hotel Name
14,guasave,mx,25.516607,-108.557260,
30,east london,za,-66.912550,48.225462,
43,guerrero negro,mx,12.890901,-130.441015,
102,alice springs,au,-26.499019,135.364132,
122,marawi,sd,19.028539,32.922236,
124,auki,sb,-2.729819,163.718290,
141,ghatanji,in,20.043351,78.511987,
174,castro,cl,-50.276241,-89.312267,
227,ballina,au,-31.149316,158.836065,
282,kirakira,sb,-15.131528,162.232037,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters
# radius has been increased to 50 km, because no hotels could be found within a 5 km radius
params = {
    "radius": 50000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # Get lat, lng data from each row and use that as a location parameter for the API
    lat = row["Latitude"]
    lng = row["Longtitude"]
    
    params["location"] = f'{lat},{lng}'
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        
    except:
        # If no hotel can be found within a 50 km radius, add "N/A" as value
        hotel_df.loc[index, "Hotel Name"] = "N/A"

hotel_df

,City,Country,Latitude,Longtitude,Hotel Name
14,guasave,mx,25.516607,-108.557260,El Sembrador Hotel
30,east london,za,-66.912550,48.225462,N/A
43,guerrero negro,mx,12.890901,-130.441015,N/A
102,alice springs,au,-26.499019,135.364132,3 O'clock Creek
122,marawi,sd,19.028539,32.922236,N/A
124,auki,sb,-2.729819,163.718290,N/A
141,ghatanji,in,20.043351,78.511987,Dwaraka Hotel.
174,castro,cl,-50.276241,-89.312267,N/A
227,ballina,au,-31.149316,158.836065,Beachcomber Lodge
282,kirakira,sb,-15.131528,162.232037,N/A


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longtitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))